In [1]:
#Import your libraries

import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')
import pylab as plt
import seaborn as sns
import difflib
#%pip install fuzzywuzzy
from fuzzywuzzy import fuzz

In [2]:
obras_completo_original = pd.read_csv('/Users/karmelealonsoaia/Desktop/ironhack_labs/PROYECTOS/project_final/data/sparql_2023-11-30_10-13-22Z.csv')

In [3]:
dim_original = obras_completo_original.shape # De esta forma guardo las dimensiones del data frame original.
obras_completo_original.shape

(655, 12)

In [4]:
obras_original = obras_completo_original.copy()

In [5]:
# Estudio de la información de iconografía en el Museo Del Prado.

In [6]:
# 1. Exploración inicial.

In [7]:
obras_original.head()

,uriObra,tituloObra,autores,fecha,anio,Escuelas,Objetos,Faunas,Floras,LugaresGeo,Personajes,Imagenes
0,http://museodelprado.es/items/E22_Man-Made_Obj...,Floreros,"Nuzzi, Mario || Nuzzi, Mario (Mario dei Fiori)",1640 - 1642,1640,Italiana,Enseres domésticos || Florero || Instrumentos ...,NaN,Anémona (Anemone spp.) || Gladiolo (Gladiolus ...,NaN,NaN,https://content3.cdnprado.net/imagenes/Documen...
1,http://museodelprado.es/items/E22_Man-Made_Obj...,La Anunciación,Maestro de la Sisla || Master of La Sisla,Hacia 1500,1495,Española,Libro,NaN,Azucena (Lilium candidum),NaN,Virgen María,https://content3.cdnprado.net/imagenes/Documen...
2,http://museodelprado.es/items/E22_Man-Made_Obj...,Copa de la Vendimia,"Fontana, Annibale || Taller de los Sarachi || ...",1570 - 1600,1570,Italiana,NaN,NaN,Vid / Parra (Vitis vinifera),NaN,NaN,https://content3.cdnprado.net/imagenes/Documen...
3,http://museodelprado.es/items/E22_Man-Made_Obj...,Florero,"Pérez, Bartolomé",Hacia 1676,1671,Española,NaN,Artrópodos (Filo: Arthropoda),Ajenuz / Arañuela (Nigella damascena) || Anémo...,NaN,NaN,https://content3.cdnprado.net/imagenes/Documen...
4,http://museodelprado.es/items/E22_Man-Made_Obj...,La Inmaculada Concepción,"Maella, Mariano Salvador",Hacia 1781,1776,Española,Elementos heráldicos y nobiliarios || Insignia...,NaN,Azucena (Lilium candidum),NaN,Virgen María || Ángel / ángeles,https://content3.cdnprado.net/imagenes/Documen...


In [8]:
obras_original.columns # Esto nos proporciona una enumeración de los nombres correspondientes a cada una de las columnas presentes en los datos.

Index(['uriObra', 'tituloObra', 'autores', 'fecha', 'anio', 'Escuelas',
       'Objetos', 'Faunas', 'Floras', 'LugaresGeo', 'Personajes', 'Imagenes'],
      dtype='object')

In [9]:
obras_original.info() # De este modo conzco el tipo de dato y los valores nulos que tiene cada una de las columnas.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 655 entries, 0 to 654
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   uriObra     655 non-null    object
 1   tituloObra  655 non-null    object
 2   autores     655 non-null    object
 3   fecha       655 non-null    object
 4   anio        655 non-null    int64 
 5   Escuelas    649 non-null    object
 6   Objetos     319 non-null    object
 7   Faunas      188 non-null    object
 8   Floras      655 non-null    object
 9   LugaresGeo  48 non-null     object
 10  Personajes  309 non-null    object
 11  Imagenes    655 non-null    object
dtypes: int64(1), object(11)
memory usage: 61.5+ KB


In [10]:
# Renombrar las columnas
obras_original = obras_original.rename(columns={
    'uriObra': 'Obra_uri',
    'tituloObra': 'Titulo',
    'autores': 'Autor',
    'fecha': 'Fecha',
    'anio': 'Año',
    'Floras': 'Flora',
    'Escuelas': 'Escuela',
    'Faunas': 'Fauna',
    'LugaresGeo': 'Lugar',
})

In [11]:
columnas_a_procesar = ['Autor', 'Flora', 'Imagenes', 'Escuela', 'Objetos', 'Fauna', 'Lugar', 'Personajes']

# Reemplazar "||" con ";" y mantener los valores nulos
obras_original[columnas_a_procesar] = obras_original[columnas_a_procesar].applymap(lambda x: x.replace('||', ';') if isinstance(x, str) else x)


In [12]:
#Antes de nada, voy a quitar las columnas que más pesan para hacer el estudio, las guardaremos dentro de otros csv por si luego queremos acudir a ellas.

In [13]:
obras_completo = obras_original[["Titulo", "Autor", "Fecha", "Año", "Escuela", "Objetos", "Personajes", "Flora", "Fauna", "Lugar"]]
uri_imagenes_obras = obras_original[["Titulo", "Obra_uri", "Imagenes"]]

# Análisis de "Año" y "Fecha"

In [14]:
obras_completo[['Fecha', 'Año']]

,Fecha,Año
0,1640 - 1642,1640
1,Hacia 1500,1495
2,1570 - 1600,1570
3,Hacia 1676,1671
4,Hacia 1781,1776
...,...,...
650,Antes de 1649,1629
651,1876,1876
652,Hacia 1520,1515
653,Primera mitad del siglo XVII,1601


In [15]:
def transformar_años(año):
    if año == -10:
        return -100
    elif año == -20:
        return -200
    elif año == -30:
        return -300
    elif año == -40:
        return -400
    else:
        return año
      

obras_completo['Año'] = obras_completo['Año'].apply(transformar_años)

In [16]:
obras_completo['Año'].unique()

array([1640, 1495, 1570, 1671, 1776, 1601, 1924, 1856, 1727, 1615, 1663,
       1505, 1642, 1908, 1832, 1660, 1689, 1550, 1874, 1650, 1685,  -21,
       1787, 1635, 1859, 1501, 1515, 1545, 1906, 1490, 1885, 1585, 1659,
       1867, 1551, 1799, 1805, 1826, 1425, 1829, 1595, 1783, 1654, 1780,
       1847, 1670, 1901, 1919, 1628, 1779, 1620, 1751, 1872, 1846, 1634,
       1769, 1633, 1857, 1643, 1630, 1655, 1913, 1804, 1631, 1600, 1868,
       1870, 1915, 1603, 1900, 1811, 1541, 1910, 1658, 1795, 1692, 1627,
       1651, 1803, 1724, 1875, 1423, 1624, 1871, 1869, 1888, 1675, 1695,
       1680, 1690, 1719, 1567,   50, 1701, 1770, 1890, 1790, 1876, 1694,
       1610, 1785, 1898, 1914, 1858, 1882, 1881, 1844, 1616, 1806, 1668,
       1612, 1784, -100, 1866, 1877, 1879, 1862, 1678, 1676, 1617, 1887,
       1813, 1638, 1531, 1491, 1664, 1745, 1781, -300, 1892, 1665, 1606,
       1775, 1465, 1644, 1604, 1860, 1509, 1895, 1789, 1764, 1760, 1884,
       1837, 1774, 1480, 1801, 1969, 1645, 1842, 18

In [17]:
#Puede verse que un dato cuerioso es el -21, vamos a ver que ocurre ahí:

In [18]:
obras_completo.loc[obras_completo['Año'] == -21]

,Titulo,Autor,Fecha,Año,Escuela,Objetos,Personajes,Flora,Fauna,Lugar
28,Hércules / Palmera con frutos,Anonymous ; Anónimo,Finales del siglo III a.C. - Primera mitad del...,-21,Hispano-cartaginesa,NaN,Hércules/Heracles,Palmera (Phoenix dactylifera),NaN,NaN


In [19]:
#en efecto, coincide con el siglo III a.C, por lo que lo modificaremos por un -210.

In [20]:
obras_completo['Año'] = obras_completo['Año'].replace(-21, -210)

In [21]:
def transformar_fechas(fecha):
    dic_fechas = {
        'Primer cuarto del siglo XVII': '1601 - 1625',
        'Siglo XVII': '1601 - 1700',
        'Finales del siglo XVII - Principio del siglo XVIII': '1685 - 1715',
        'Finales del siglo III a.C. - Primera mitad del siglo II a.C.': '-30 - -25',
        'Siglo XVI': '1501 - 1600',
        'Segunda mitad del siglo XVI': '1551 - 1600',
        'Principio del siglo XX': '1901 - 1915',
        'Último tercio del siglo XVIII': '1769 - 1800',
        'Finales del siglo XVI - Primer tercio del siglo XVII': '1585 - 1630',
        'Primera mitad del siglo XVII': '1601 - 1650',
        'Segunda mitad del siglo XVII': '1651 - 1700',
        'Finales del siglo XVII': '1685 - 1700',
        'Siglo XVIII': '1701 - 1800',
        'Finales del siglo XVIII': '1785 - 1800',
        'Siglos XVI - XVII': '1501 - 1700',
        'Segunda mitad del siglo XVII - Primer cuarto del siglo XVIII': '1651 - 1725',
        'Siglo I a.C.': '-100',
        'Finales del siglo XVI': '1685 - 1700',
        'Primer tercio del siglo XVII': '1601 - 1630',
        'Siglo III a.C.': '-300',
        'Primer tercio del siglo XVI': '1501 - 1530',
        'Segundo tercio del siglo XVII': '1634 - 1870',
        'Siglo XII': '1101 - 1199',
        'Último cuarto del siglo XVIII - Principio del siglo XIX': '1776 - 1815',
        'Último cuarto del siglo XVII': '1676 - 1700',
        'Siglos IV a.C. - III a.C.': '-400 - -300',
        'Mediados del siglo XVII': '1640 - 1660',
        'Siglo XIX': '1801 - 1900',
    }

    return dic_fechas.get(fecha, fecha)

obras_completo['rango_fecha'] = obras_completo['Fecha'].apply(transformar_fechas)

In [22]:
# Si 'Hacia' está presente en la columna 'Fecha', tomar el valor de la columna 'Año'; sino, mantener el valor de la columna 'Fecha'

def asignar_rango_fecha(celda):
    if 'Hacia' in celda['Fecha']:
        return celda['Año']
    else:
        return celda['Fecha']

# Aplicar la función a cada celda del DataFrame y almacenar el resultado en la nueva columna 'Fecha_rango'
obras_completo['rango_fecha'] = obras_completo.apply(asignar_rango_fecha, axis=1)

# Análisis "Autor"

In [23]:
obras_completo.head()

,Titulo,Autor,Fecha,Año,Escuela,Objetos,Personajes,Flora,Fauna,Lugar,rango_fecha
0,Floreros,"Nuzzi, Mario ; Nuzzi, Mario (Mario dei Fiori)",1640 - 1642,1640,Italiana,Enseres domésticos ; Florero ; Instrumentos de...,NaN,Anémona (Anemone spp.) ; Gladiolo (Gladiolus s...,NaN,NaN,1640 - 1642
1,La Anunciación,Maestro de la Sisla ; Master of La Sisla,Hacia 1500,1495,Española,Libro,Virgen María,Azucena (Lilium candidum),NaN,NaN,1495
2,Copa de la Vendimia,"Fontana, Annibale ; Taller de los Sarachi ; Wo...",1570 - 1600,1570,Italiana,NaN,NaN,Vid / Parra (Vitis vinifera),NaN,NaN,1570 - 1600
3,Florero,"Pérez, Bartolomé",Hacia 1676,1671,Española,NaN,NaN,Ajenuz / Arañuela (Nigella damascena) ; Anémon...,Artrópodos (Filo: Arthropoda),NaN,1671
4,La Inmaculada Concepción,"Maella, Mariano Salvador",Hacia 1781,1776,Española,Elementos heráldicos y nobiliarios ; Insignia ...,Virgen María ; Ángel / ángeles,Azucena (Lilium candidum),NaN,NaN,1776


In [24]:
nombres_unicos = obras_completo['Autor'].unique()
nombres_unicos = list(nombres_unicos)
nombres_unicos

['Nuzzi, Mario ; Nuzzi, Mario (Mario dei Fiori)',
 'Maestro de la Sisla ; Master of La Sisla',
 'Fontana, Annibale ; Taller de los Sarachi ; Workshop of the Sarachi',
 'Pérez, Bartolomé',
 'Maella, Mariano Salvador',
 "Brueghel el Viejo, Jan ; Brueghel the Elder, Jan (Jan 'Velvet' Brueghel)",
 'Martí Garcés de Marcilla, José',
 'Haes, Carlos de',
 'Tobar, Alonso Miguel de',
 'Balen, Hendrick van ; Brueghel el Joven, Jan ; Brueghel the Younger, Jan',
 'Camprobín, Pedro de',
 'Rincón, Fernando del',
 'Hiepes, Tomás',
 'Bertodano, Luis de',
 'Guglielmi, Augusto ; Madrazo y Agudo, José de ; Real Establecimiento Litográfico de Madrid',
 'Vauquer, Jacques',
 'Coosemans, Alexander',
 'Leoni, Leone',
 'J. Laurent y Cía',
 'Arellano, Juan de',
 'Galleria dei Lavori, Florencia',
 'Belvedere, Andrea ; Solimena, Francesco',
 'Anonymous ; Anónimo',
 'Malaine, Joseph-Laurent',
 'Dyck, Anthony van ; Dyck, Anton van',
 'Puebla y Tolín, Dióscoro Teófilo',
 'Maestro de la Santa Sangre',
 'David, Gérard'

In [25]:
for e in nombres_unicos:
    print(e)

Nuzzi, Mario ; Nuzzi, Mario (Mario dei Fiori)
Maestro de la Sisla ; Master of La Sisla
Fontana, Annibale ; Taller de los Sarachi ; Workshop of the Sarachi
Pérez, Bartolomé
Maella, Mariano Salvador
Brueghel el Viejo, Jan ; Brueghel the Elder, Jan (Jan 'Velvet' Brueghel)
Martí Garcés de Marcilla, José
Haes, Carlos de
Tobar, Alonso Miguel de
Balen, Hendrick van ; Brueghel el Joven, Jan ; Brueghel the Younger, Jan
Camprobín, Pedro de
Rincón, Fernando del
Hiepes, Tomás
Bertodano, Luis de
Guglielmi, Augusto ; Madrazo y Agudo, José de ; Real Establecimiento Litográfico de Madrid
Vauquer, Jacques
Coosemans, Alexander
Leoni, Leone
J. Laurent y Cía
Arellano, Juan de
Galleria dei Lavori, Florencia
Belvedere, Andrea ; Solimena, Francesco
Anonymous ; Anónimo
Malaine, Joseph-Laurent
Dyck, Anthony van ; Dyck, Anton van
Puebla y Tolín, Dióscoro Teófilo
Maestro de la Santa Sangre
David, Gérard
Titian (Tiziano Vecellio) ; Tiziano, Vecellio di Gregorio
Pulido Fernández, Ramón
Pereda y Salgado, Antonio de

In [26]:
nombres_unicos[0]

'Nuzzi, Mario ; Nuzzi, Mario (Mario dei Fiori)'

In [27]:
descomposicion_de_nombres = nombres_unicos[0].split(";")
descomposicion_de_nombres

['Nuzzi, Mario ', ' Nuzzi, Mario (Mario dei Fiori)']

In [28]:
def elegir_autor(nombres):
    # Divide la cadena de nombres en una lista y elimina los espacios alrededor de cada nombre
    lista_nombres = [nombre.strip() for nombre in nombres.split(';')]
    
    # Calcula la similitud entre los conjuntos de palabras
    puntajes_similitud = []
    for i in range(len(lista_nombres)):
        for j in range(i + 1, len(lista_nombres)):
            puntaje = fuzz.token_set_ratio(lista_nombres[i], lista_nombres[j])
            puntajes_similitud.append((i, j, puntaje))
    
    # Ordena la lista de similitudes de mayor a menor
    puntajes_similitud.sort(key=lambda x: x[2], reverse=True)
    
    # Devuelve el nombre con mayor similitud si supera el umbral del 50%
    if puntajes_similitud and puntajes_similitud[0][2] > 50:  # Puedes ajustar el umbral según tus necesidades
        return lista_nombres[puntajes_similitud[0][0]]
    else:
        return nombres

In [29]:
obras_completo['Autor'] = obras_completo['Autor'].apply(elegir_autor)

In [30]:
obras_completo.head()

,Titulo,Autor,Fecha,Año,Escuela,Objetos,Personajes,Flora,Fauna,Lugar,rango_fecha
0,Floreros,"Nuzzi, Mario",1640 - 1642,1640,Italiana,Enseres domésticos ; Florero ; Instrumentos de...,NaN,Anémona (Anemone spp.) ; Gladiolo (Gladiolus s...,NaN,NaN,1640 - 1642
1,La Anunciación,Maestro de la Sisla,Hacia 1500,1495,Española,Libro,Virgen María,Azucena (Lilium candidum),NaN,NaN,1495
2,Copa de la Vendimia,Taller de los Sarachi,1570 - 1600,1570,Italiana,NaN,NaN,Vid / Parra (Vitis vinifera),NaN,NaN,1570 - 1600
3,Florero,"Pérez, Bartolomé",Hacia 1676,1671,Española,NaN,NaN,Ajenuz / Arañuela (Nigella damascena) ; Anémon...,Artrópodos (Filo: Arthropoda),NaN,1671
4,La Inmaculada Concepción,"Maella, Mariano Salvador",Hacia 1781,1776,Española,Elementos heráldicos y nobiliarios ; Insignia ...,Virgen María ; Ángel / ángeles,Azucena (Lilium candidum),NaN,NaN,1776


In [31]:
# Filtrar las filas donde la columna 'Autor' contiene un punto y coma
autor_puntocoma = obras_completo[obras_completo['Autor'].str.contains(';')]['Autor']

# Obtener una lista de valores únicos
print(autor_puntocoma.unique())

['Guglielmi, Augusto ; Madrazo y Agudo, José de ; Real Establecimiento Litográfico de Madrid'
 'Belvedere, Andrea ; Solimena, Francesco'
 'Blanco y Assensio, Alejandro ; Madrazo y Agudo, José de ; Real Establecimiento Litográfico de Madrid'
 'Legrand, Luis Carlos ; Madrazo y Agudo, José de ; Real Establecimiento Litográfico de Madrid'
 'Decraene, Florentino ; Madrazo y Agudo, José de ; Real Establecimiento Litográfico de Madrid'
 'López, Juan Antonio ; Madrazo y Agudo, José de ; Real Establecimiento Litográfico de Madrid'
 'Quellinus, Erasmus ; Seghers, Daniel' 'Bosch, Hieronymus ; El Bosco'
 'Belvedere, Andrea ; Giordano, Luca'
 'Flémalle, Bertholet ; Legrand, Luis Carlos ; Madrazo y Agudo, José de ; Poussin, Nicolas ; Real Establecimiento Litográfico de Madrid'
 'Madrazo y Agudo, José de ; Real Establecimiento Litográfico de Madrid ; Rodríguez, Cayetano'
 'Decraene, Florentino ; Madrazo y Agudo, José de ; Murillo, Bartolomé Esteban ; Real Establecimiento Litográfico de Madrid'
 'Calc

In [32]:
# Reemplazar 'Anonymous' por 'Anonimo' en la columna 'Autor'
obras_completo['Autor'] = obras_completo['Autor'].replace('Anonymous', 'Anonimo')

# Análisis "Escuela"

In [33]:
obras_completo['Escuela'].unique()

array(['Italiana', 'Española', 'Flamenca', 'Francesa',
       'Hispano-cartaginesa', 'Italiana ; Veneciana',
       'Castellana ; Española', 'Francesa ; Italiana', 'Holandesa',
       'Alemana', 'Española ; Madrileña', 'Italiana ; Napolitana',
       'Arte Clásico ; Roma', 'Boloñesa ; Italiana', 'Gala', nan,
       'Púnica', 'Hispano-flamenca', 'Española ; Valenciana',
       'Florentina ; Italiana', 'Inglesa', 'Sueca',
       'Castellana ; Española ; Toledana',
       'Francesa ; Italiana ; Milanesa', 'Portuguesa'], dtype=object)

In [34]:
obras_completo['Escuela'].isna().any()

True

In [35]:
obras_completo['Escuela'] = obras_completo['Escuela'].fillna('Desconocida')

In [36]:
obras_completo['Escuela'].unique()

array(['Italiana', 'Española', 'Flamenca', 'Francesa',
       'Hispano-cartaginesa', 'Italiana ; Veneciana',
       'Castellana ; Española', 'Francesa ; Italiana', 'Holandesa',
       'Alemana', 'Española ; Madrileña', 'Italiana ; Napolitana',
       'Arte Clásico ; Roma', 'Boloñesa ; Italiana', 'Gala',
       'Desconocida', 'Púnica', 'Hispano-flamenca',
       'Española ; Valenciana', 'Florentina ; Italiana', 'Inglesa',
       'Sueca', 'Castellana ; Española ; Toledana',
       'Francesa ; Italiana ; Milanesa', 'Portuguesa'], dtype=object)

In [37]:
#En el análisis subsiguiente, no tiene sentido alguno hacer un estudio en el que se comparen la escuela toledana, por ejemplo, con la española, pues una implica a la otra, por este motivo, dejaremos, al igual que en Lugares, unicamente las escuelas mas generales.

In [38]:
def modificar_escuela(escuela):
    palabras_clave = ['Española', 'Italiana', 'Francesa']
    
    # Verifica si cada palabra clave está en la cadena y devuelve la primera encontrada
    for palabra in palabras_clave:
        if palabra in escuela:
            return palabra
    
    # Si no se encuentra ninguna palabra clave, devuelve la cadena original
    return escuela

# Aplicar la función a la columna 'Escuela'
obras_completo['Escuela'] = obras_completo['Escuela'].apply(modificar_escuela)

In [39]:
obras_completo['Escuela'].unique()

array(['Italiana', 'Española', 'Flamenca', 'Francesa',
       'Hispano-cartaginesa', 'Holandesa', 'Alemana',
       'Arte Clásico ; Roma', 'Gala', 'Desconocida', 'Púnica',
       'Hispano-flamenca', 'Inglesa', 'Sueca', 'Portuguesa'], dtype=object)

In [40]:
# Modificar la columna 'Escuela' después de obtener los resultados anteriores
obras_completo['Escuela'] = obras_completo['Escuela'].replace('Arte Clásico ; Roma', 'Arte Clásico')


In [41]:
obras_completo.head()

,Titulo,Autor,Fecha,Año,Escuela,Objetos,Personajes,Flora,Fauna,Lugar,rango_fecha
0,Floreros,"Nuzzi, Mario",1640 - 1642,1640,Italiana,Enseres domésticos ; Florero ; Instrumentos de...,NaN,Anémona (Anemone spp.) ; Gladiolo (Gladiolus s...,NaN,NaN,1640 - 1642
1,La Anunciación,Maestro de la Sisla,Hacia 1500,1495,Española,Libro,Virgen María,Azucena (Lilium candidum),NaN,NaN,1495
2,Copa de la Vendimia,Taller de los Sarachi,1570 - 1600,1570,Italiana,NaN,NaN,Vid / Parra (Vitis vinifera),NaN,NaN,1570 - 1600
3,Florero,"Pérez, Bartolomé",Hacia 1676,1671,Española,NaN,NaN,Ajenuz / Arañuela (Nigella damascena) ; Anémon...,Artrópodos (Filo: Arthropoda),NaN,1671
4,La Inmaculada Concepción,"Maella, Mariano Salvador",Hacia 1781,1776,Española,Elementos heráldicos y nobiliarios ; Insignia ...,Virgen María ; Ángel / ángeles,Azucena (Lilium candidum),NaN,NaN,1776


In [42]:
nuevo_orden = ["Titulo", "Autor", "Fecha", "Año", "Escuela", "Objetos", "Fauna", "Flora", "Lugar", "Personajes", "rango_fecha"]
obras_completo = obras_completo.reindex(columns=nuevo_orden)

# Análisis "Lugar"

In [43]:
#voy a hacer, por el mimso motivo que en escuela, lo mismo para lugar, no tiene sentido hacer un estudio del mismo modo que àra el resto de iconografía en el que se comparen países, ciudades, etc.

In [44]:
def modificar_lugar(lugar):
    if pd.notna(lugar):  # Verifica si no es NaN
        palabras_lugar = ['España', 'Italia', 'Francia', 'Marruecos', 'Bélgica']
        
        # Verifica si cada palabra clave está en la cadena y devuelve la primera encontrada
        for palabra in palabras_lugar:
            if palabra in lugar:
                return palabra
    
    # Si no se encuentra ninguna palabra clave o es NaN, devuelve la cadena original o NaN
    return lugar

# Aplicar la función a la columna 'Lugar'
obras_completo['Lugar'] = obras_completo['Lugar'].apply(modificar_lugar)


In [45]:
obras_completo['Lugar'].unique()

array([nan, 'España', 'Italia', 'Francia', 'Marruecos', 'Bélgica',
       'Argelia'], dtype=object)

In [46]:
obras_estudio = obras_completo.copy()

# Crear ID

In [47]:
obras_estudio['TituloID'] = range(len(obras_estudio))

In [48]:
nuevo_orden = ["TituloID", "Titulo", "Autor", "Fecha", "Año", "Escuela", "Objetos", "Fauna", "Flora", "Lugar", "Personajes", "rango_fecha"]
obras_estudio = obras_estudio.reindex(columns=nuevo_orden)

# Creación de DataFrames para estudio

In [49]:
obras_estudio.head(2)

,TituloID,Titulo,Autor,Fecha,Año,Escuela,Objetos,Fauna,Flora,Lugar,Personajes,rango_fecha
0,0,Floreros,"Nuzzi, Mario",1640 - 1642,1640,Italiana,Enseres domésticos ; Florero ; Instrumentos de...,NaN,Anémona (Anemone spp.) ; Gladiolo (Gladiolus s...,NaN,NaN,1640 - 1642
1,1,La Anunciación,Maestro de la Sisla,Hacia 1500,1495,Española,Libro,NaN,Azucena (Lilium candidum),NaN,Virgen María,1495


In [50]:
obras = obras_estudio[["TituloID", "Titulo", "Autor", "Fecha", "Año", "Escuela", "rango_fecha"]]
objetos = obras_estudio[["TituloID", "Objetos"]]
personajes = obras_estudio[["TituloID", "Personajes"]]
flora = obras_estudio[["TituloID", "Flora"]]
fauna = obras_estudio[["TituloID", "Fauna"]]
lugar = obras_estudio[["TituloID", "Lugar"]]

In [51]:
obras['Autor'] = obras['Autor'].str.split(';')
obras = obras.explode('Autor')
obras['Autor'] = obras['Autor'].str.strip()

In [52]:
personajes['Personajes'] = personajes['Personajes'].str.split(';')
personajes = personajes.explode('Personajes')
personajes['Personajes'] = personajes['Personajes'].str.strip()

In [53]:
objetos['Objetos'] = objetos['Objetos'].str.split(';')
objetos = objetos.explode('Objetos')
objetos['Objetos'] = objetos['Objetos'].str.strip()

In [54]:
flora['Flora'] = flora['Flora'].str.split(';')
flora = flora.explode('Flora')
flora['Flora'] = flora['Flora'].str.strip()

In [55]:
fauna['Fauna'] = fauna['Fauna'].str.split(';')
fauna = fauna.explode('Fauna')
fauna['Fauna'] = fauna['Fauna'].str.strip()

In [56]:
obras_estudio.head()

,TituloID,Titulo,Autor,Fecha,Año,Escuela,Objetos,Fauna,Flora,Lugar,Personajes,rango_fecha
0,0,Floreros,"Nuzzi, Mario",1640 - 1642,1640,Italiana,Enseres domésticos ; Florero ; Instrumentos de...,NaN,Anémona (Anemone spp.) ; Gladiolo (Gladiolus s...,NaN,NaN,1640 - 1642
1,1,La Anunciación,Maestro de la Sisla,Hacia 1500,1495,Española,Libro,NaN,Azucena (Lilium candidum),NaN,Virgen María,1495
2,2,Copa de la Vendimia,Taller de los Sarachi,1570 - 1600,1570,Italiana,NaN,NaN,Vid / Parra (Vitis vinifera),NaN,NaN,1570 - 1600
3,3,Florero,"Pérez, Bartolomé",Hacia 1676,1671,Española,NaN,Artrópodos (Filo: Arthropoda),Ajenuz / Arañuela (Nigella damascena) ; Anémon...,NaN,NaN,1671
4,4,La Inmaculada Concepción,"Maella, Mariano Salvador",Hacia 1781,1776,Española,Elementos heráldicos y nobiliarios ; Insignia ...,NaN,Azucena (Lilium candidum),NaN,Virgen María ; Ángel / ángeles,1776


In [57]:
obras_estudio = pd.merge(obras, objetos, on='TituloID', how='inner')

In [58]:
obras_estudio = pd.merge(obras_estudio, personajes, on='TituloID', how='inner')

In [59]:
obras_estudio = pd.merge(obras_estudio, flora, on='TituloID', how='inner')

In [60]:
obras_estudio = pd.merge(obras_estudio, fauna, on='TituloID', how='inner')

In [61]:
obras_estudio = pd.merge(obras_estudio, lugar, on='TituloID', how='inner')

In [62]:
obras_estudio.duplicated().any()

False

In [63]:
nuevo_orden = ["TituloID", "Titulo", "Autor", "Fecha", "Año", "Escuela", "Objetos", "Personajes", "Flora", "Fauna", "Lugar", "rango_fecha"]
obras_estudio = obras_estudio.reindex(columns=nuevo_orden)

In [64]:
obras_estudio = obras_estudio.drop('TituloID', axis=1)

In [65]:
obras_estudio.head()

,Titulo,Autor,Fecha,Año,Escuela,Objetos,Personajes,Flora,Fauna,Lugar,rango_fecha
0,Floreros,"Nuzzi, Mario",1640 - 1642,1640,Italiana,Enseres domésticos,NaN,Anémona (Anemone spp.),NaN,NaN,1640 - 1642
1,Floreros,"Nuzzi, Mario",1640 - 1642,1640,Italiana,Enseres domésticos,NaN,Gladiolo (Gladiolus spp.),NaN,NaN,1640 - 1642
2,Floreros,"Nuzzi, Mario",1640 - 1642,1640,Italiana,Enseres domésticos,NaN,Lirio (Iris germanica),NaN,NaN,1640 - 1642
3,Floreros,"Nuzzi, Mario",1640 - 1642,1640,Italiana,Enseres domésticos,NaN,Mundillo / Bola de nieve (Viburnum opulus L.),NaN,NaN,1640 - 1642
4,Floreros,"Nuzzi, Mario",1640 - 1642,1640,Italiana,Enseres domésticos,NaN,Peonía (Paeonia),NaN,NaN,1640 - 1642


In [66]:
obras_estudio['Lugar'].unique()

array([nan, 'España', 'Italia', 'Francia', 'Marruecos', 'Bélgica',
       'Argelia'], dtype=object)

In [68]:
obras_estudio.shape

(25735, 11)

# Exportación de los datframes

In [69]:
obras_estudio.to_csv("/Users/karmelealonsoaia/Desktop/ironhack_labs/PROYECTOS/project_final/data/data_def/obras_estudio.csv", index=False)